## Comparison between the iterative Image Space Restoration Algorithm (ISRA) and the Richardson-Lucy Algorithm (RLA) 
Experimental data

In [1]:
#required libraries
%matplotlib qt
import numpy as np
import hyperspy.api as hs
from ncempy.io import dm
import matplotlib.pyplot as plt

Functions

In [2]:
#Normalize spectrum based on total electron counts
def normalizeS(S):
	S_norm = S/np.sum(S)
	return S_norm

#ISRA algorithm
def ISRA(iterations, PSF, Spec):
    ISRA5 = np.copy(Spec)
    ISRA1 = np.convolve(PSF, Spec, mode='same')
    for i in range(iterations):
        ISRA2 = np.convolve(PSF, ISRA5, mode='same')
        ISRA3 = np.convolve(PSF, ISRA2, mode='same')
        ISRA4 = np.divide(ISRA1, ISRA3)
        ISRA5 = np.multiply(ISRA4, ISRA5)
    return ISRA5

#Richardson-Lucy algorithm (from Edson Bellido)
def RL(iterations, PSF, Spec):
    RL4 = np.copy(Spec)
    for i in range(iterations):
        RL1 = np.convolve(PSF, RL4, mode='same')
        RL2 = np.divide(Spec,RL1)
        RL3 = np.convolve(PSF, RL2, mode='same')
        RL4 = np.multiply(RL3, RL4)
    return RL4

Loading files

In [4]:
#load experimental spectrum images
dataset = dm.dmReader("D:\Documents\Summer 2020 Research\Sample Data\SI_2.dm4")
psf = dm.dmReader("D:\Documents\Summer 2020 Research\Sample Data\psf.dm4")

#index for separate spectra of the same size
Spec = dataset['data'][:1778,0,0]
PSF = psf['data'][:,0,0]

print(Spec)

[257.7301  253.94647 274.17886 ... 693.9939  654.3498  684.17535]


In [5]:
#normalize
PSF_norm = normalizeS(PSF)
Spec_norm = normalizeS(Spec)

In [6]:
#perform deconvolution
ISRA = ISRA(20, PSF_norm, Spec_norm)
RL = RL(20, PSF_norm, Spec_norm)

Plot

In [7]:
fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
ax1.set_title("Spec")
ax2.set_title("PSF")
ax1.plot(Spec_norm)
ax2.plot(PSF_norm)

fig_1 = plt.figure()
ax_test1 = fig_1.add_subplot(111)
ax_test1.plot(abs(ISRA))
fig_1.suptitle("ISRA")
ax_test1.set_ylim(0,0.02)

fig_2 = plt.figure()
ax_test2 = fig_2.add_subplot(111)
ax_test2.plot(abs(RL))
fig_2.suptitle("RL")
ax_test2.set_ylim(0,0.02)

plt.show()